In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os
import sys

sys.path.append('/home/oldrain123/MMD/')
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [3]:
from tqdm.auto import tqdm
import numpy as np
import jax.numpy as jnp

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

from utils_HD import MatConvert, MMDu, TST_MMD_u

In [4]:
np.random.seed(819)
torch.manual_seed(819)
torch.cuda.manual_seed(819)
torch.backends.cudnn.deterministic = True
is_cuda = True

In [5]:
# Parameter Settings
n_epochs = 1000
batch_size = 512
lr = 0.00002
img_size = 64
channels = 3
n = 1000

In [6]:
dtype = torch.float
device = torch.device("cuda:0")
cuda = True if torch.cuda.is_available() else False
N_per = 100 # permutation times
alpha = 0.05 # test threshold
N1 = n # number of samples in one set
K = 10 # number of trails
J = 1 # number of test locations
N = 100 # number of test sets
N_f = 100.0 # number of test sets (float)

In [7]:
# Naming variables
ep_OPT = np.zeros([K])
s_OPT = np.zeros([K])
s0_OPT = np.zeros([K])
T_org_OPT = torch.zeros([K,J,3,64,64]) # Record test locations obtained by MMD-D
DK_Results = np.zeros([1,K])

In [8]:
# Define the deep network for MMD-D
class Featurizer(nn.Module):
    def __init__(self):
        super(Featurizer, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0)] #0.25
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
            # *discriminator_block(128, 256), 
            
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(
            nn.Linear(128 * ds_size ** 2, 300))

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        feature = self.adv_layer(out)

        return feature

In [9]:
# Configure data loader
dataset_test = datasets.CIFAR10(root='/data4/oldrain123/C2ST/data/cifar_data/cifar10', download=True,train=False,
                           transform=transforms.Compose([
                               transforms.Resize(img_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=10000,
                                             shuffle=True, num_workers=1)
# Obtain CIFAR10 images
for i, (imgs, Labels) in enumerate(dataloader_test):
    data_all = imgs
    label_all = Labels
Ind_all = np.arange(len(data_all))

# Obtain CIFAR10.1 images
data_new = np.load('/data4/oldrain123/C2ST/data/cifar10_1/cifar10.1_v4_data.npy')
data_T = np.transpose(data_new, [0,3,1,2])
ind_M = np.random.choice(len(data_T), len(data_T), replace=False)
data_T = data_T[ind_M]
TT = transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trans = transforms.ToPILImage()
data_trans = torch.zeros([len(data_T),3,img_size,img_size])
data_T_tensor = torch.from_numpy(data_T)
for i in range(len(data_T)):
    d0 = trans(data_T_tensor[i])
    data_trans[i] = TT(d0)
Ind_v4_all = np.arange(len(data_T))

Files already downloaded and verified


In [10]:
# Repeat experiments K times (K = 10) and report average test power (rejection rate)
for kk in tqdm(range(K)):
    if kk > 5:
        pass
    else:
        torch.manual_seed(kk * 19 + N1)
        torch.cuda.manual_seed(kk * 19 + N1)
        np.random.seed(seed=1102 * (kk + 10) + N1)
        
        # Initialize deep networks for MMD-D (called featurizer)
        featurizer = Featurizer()
        
        # Initialize parameters
        epsilonOPT = torch.log(MatConvert(np.random.rand(1) * 10 ** (-10), device, dtype))
        epsilonOPT.requires_grad = True
        sigmaOPT = MatConvert(np.ones(1) * np.sqrt(2 * 32 * 32), device, dtype)
        sigmaOPT.requires_grad = True
        sigma0OPT = MatConvert(np.ones(1) * np.sqrt(0.005), device, dtype)
        sigma0OPT.requires_grad = True
        if cuda:
            featurizer.cuda()

        # Collect CIFAR10 images
        Ind_tr = np.random.choice(len(data_all), N1, replace=False)
        Ind_te = np.delete(Ind_all, Ind_tr)
        train_data = []
        for i in Ind_tr:
            train_data.append([data_all[i], label_all[i]])

        dataloader = torch.utils.data.DataLoader(
            train_data,
            batch_size=batch_size,
            shuffle=True,
        )

        # Collect CIFAR10.1 images
        np.random.seed(seed=819 * (kk + 9) + N1)
        Ind_tr_v4 = np.random.choice(len(data_T), N1, replace=False)
        Ind_te_v4 = np.delete(Ind_v4_all, Ind_tr_v4)
        New_CIFAR_tr = data_trans[Ind_tr_v4]
        New_CIFAR_te = data_trans[Ind_te_v4]

        # Initialize optimizers
        optimizer_F = torch.optim.Adam(list(featurizer.parameters()) + [epsilonOPT] + [sigmaOPT] + [sigma0OPT], lr=lr)

        Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

        # ----------------------------------------------------------------------------------------------------
        #  Training deep networks for MMD-D (called featurizer)
        # ----------------------------------------------------------------------------------------------------
        np.random.seed(seed=1102)
        torch.manual_seed(1102)
        torch.cuda.manual_seed(1102)
        for epoch in tqdm(range(n_epochs)):
            for i, (imgs, _) in enumerate(dataloader):
                if True:
                    ind = np.random.choice(N1, imgs.shape[0], replace=False)
                    Fake_imgs = New_CIFAR_tr[ind]
                    # Adversarial ground truths
                    valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
                    fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

                    # Configure input
                    real_imgs = Variable(imgs.type(Tensor))
                    Fake_imgs = Variable(Fake_imgs.type(Tensor))
                    X = torch.cat([real_imgs, Fake_imgs], 0)
                    Y = torch.cat([valid, fake], 0).squeeze().long()

                    # ------------------------------
                    #  Train deep network for MMD-D
                    # ------------------------------
                    # Initialize optimizer
                    optimizer_F.zero_grad()
                    # Compute output of deep network
                    modelu_output = featurizer(X)
                    # Compute epsilon, sigma and sigma_0
                    ep = torch.exp(epsilonOPT) / (1 + torch.exp(epsilonOPT))
                    sigma = sigmaOPT ** 2
                    sigma0_u = sigma0OPT ** 2
                    # Compute Compute J (STAT_u)
                    DK_TEMP = MMDu(modelu_output, imgs.shape[0], X.view(X.shape[0],-1), sigma, sigma0_u, ep, complete=False)
                    dk_hsic_xx = torch.from_numpy(np.array(DK_TEMP[3])).to(torch.float32)
                    dk_hsic_yy = torch.from_numpy(np.array(DK_TEMP[4])).to(torch.float32)
                    dk_hsic_xy = torch.from_numpy(np.array(DK_TEMP[5])).to(torch.float32)
                    
                    dk_mmd = -1 * (DK_TEMP[0])
                    dk_mmd_std = torch.sqrt(DK_TEMP[1] + 10 ** (-8))
                    
                    DK_STAT_u = torch.div(dk_mmd, dk_mmd_std) + (dk_hsic_xx + dk_hsic_yy - dk_hsic_xy)
                    if (epoch+1) % 100 == 0:
                        print("-" * 50)
                        print("epoch : ",epoch+1)
                        print("dk mmd: ", -1 * dk_mmd.item(), "dk mmd_std: ", dk_mmd_std.item(), "dk statistic: ",
                        -1 * DK_STAT_u.item())
                        
                    # Compute gradient
                    DK_STAT_u.backward()                
                    # Update weights using gradient descent
                    optimizer_F.step()
                else:
                    break

        # Run two-sample test on the training set
        # Fetch training data
        s1 = data_all[Ind_tr]
        s2 = data_trans[Ind_tr_v4]
        S = torch.cat([s1.cpu(), s2.cpu()], 0).cuda()
        Sv = S.view(2 * N1, -1)
        # Run two-sample test (MMD-D) on the training set
        # dk_h_u, dk_threshold_u, dk_mmd_value_u = TST_MMD_u(featurizer(S), N_per, N1, Sv, sigma, sigma0_u, ep, alpha, device, dtype, complete=False)
        # com_h_u, com_threshold_u, com_mmd_value_u = TST_MMD_u(featurizer(S), N_per, N1, Sv, sigma, sigma0_u, ep, alpha, device, dtype, complete=True)

        # Record best epsilon, sigma and sigma_0
        ep_OPT[kk] = ep.item()
        s_OPT[kk] = sigma.item()
        s0_OPT[kk] = sigma0_u.item()

        # Compute test power of MMD-D and baselines
        DK_H_u = np.zeros(N)

        np.random.seed(1102)
        dk_count_u = 0

        for k in tqdm(range(N)):
            # Fetch test data
            np.random.seed(seed=1102 * (k + 1) + N1)
            data_all_te = data_all[Ind_te]
            N_te = len(data_trans)-N1
            Ind_N_te = np.random.choice(len(Ind_te), N_te, replace=False)
            s1 = data_all_te[Ind_N_te]
            s2 = data_trans[Ind_te_v4]
            S = torch.cat([s1.cpu(), s2.cpu()], 0).cuda()
            Sv = S.view(2 * N_te, -1)
            # MMD-D
            dk_h_u, dk_threshold_u, dk_mmd_value_u = TST_MMD_u(featurizer(S), N_per, N_te, Sv, sigma, sigma0_u, ep, alpha, device, dtype, complete=False)

            # Gather results
            dk_count_u = dk_count_u + dk_h_u
            print("\r","MMD-DK:", dk_count_u, "MMD: ", dk_mmd_value_u, end="")
            DK_H_u[k] = dk_h_u

        # Print test power of MMD-D and baselines
        print("DK Reject rate_u: ", DK_H_u.sum() / N_f)
        DK_Results[0, kk] = DK_H_u.sum() / N_f
        print(f"Test Power of DK ({K} times): ")
        print(f"{DK_Results}")
        print(f"Average Test Power of DK ({K} times): ")
        print("MMD-D: ", (DK_Results.sum(1) / (kk + 1))[0])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/tmp/ipykernel_3865426/3968968829.py:60: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392035629/work/torch/csrc/tensor/python_tensor.cpp:83.)
  valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
CUDA backend failed to initialize: Found cuPTI version 18, but JAX was built against version 20, which is newer. The copy of cuPTI that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


mmd2:  5.3442083299160004e-05 	 var:  1.2270756997168064e-06
mmd2:  5.192245589569211e-05 	 var:  1.990687451325357e-06
mmd2:  5.914946086704731e-06 	 var:  1.0268922778777778e-06
mmd2:  9.648088598623872e-05 	 var:  2.3553584469482303e-06
mmd2:  4.830287070944905e-05 	 var:  1.4786201063543558e-06
mmd2:  4.8554560635238886e-05 	 var:  1.941465598065406e-06
mmd2:  6.739539094269276e-05 	 var:  1.5097903087735176e-06
mmd2:  7.796008139848709e-05 	 var:  2.360102371312678e-06
mmd2:  3.0437426175922155e-05 	 var:  1.094886101782322e-06
mmd2:  0.00012391043128445745 	 var:  3.544526407495141e-06
mmd2:  0.00012872030492872 	 var:  2.4683322408236563e-06
mmd2:  9.419111302122474e-05 	 var:  2.031432813964784e-06
mmd2:  9.196670725941658e-05 	 var:  1.6588965081609786e-06
mmd2:  8.279841858893633e-05 	 var:  2.6276902644895017e-06
mmd2:  0.0001607865560799837 	 var:  3.401684807613492e-06
mmd2:  9.228842100128531e-05 	 var:  1.934247848112136e-06
mmd2:  0.00010991084855049849 	 var:  3.400084

  0%|          | 0/100 [00:00<?, ?it/s]

...

mmd2:  7.856055162847042e-06 	 var:  4.5046087887803845e-11
 MMD-DK: 0 MMD:  7.856055162847042e-06mmd2:  1.0298361303284764e-05 	 var:  2.0953168618753758e-11
 MMD-DK: 1 MMD:  1.0298361303284764e-05mmd2:  2.2640248062089086e-05 	 var:  7.732780225036104e-11
 MMD-DK: 2 MMD:  2.2640248062089086e-05mmd2:  2.0423787645995617e-05 	 var:  8.178197905908156e-11
 MMD-DK: 3 MMD:  2.0423787645995617e-05mmd2:  1.9130297005176544e-05 	 var:  6.842004850789011e-11
 MMD-DK: 4 MMD:  1.9130297005176544e-05mmd2:  1.1524301953613758e-05 	 var:  4.122343529328745e-11
 MMD-DK: 5 MMD:  1.1524301953613758e-05mmd2:  1.3566634152084589e-05 	 var:  7.29417626849308e-11
 MMD-DK: 6 MMD:  1.3566634152084589e-05mmd2:  1.6292440705001354e-05 	 var:  6.19769795841755e-11
 MMD-DK: 7 MMD:  1.6292440705001354e-05mmd2:  3.279210068285465e-05 	 var:  1.7838705674870888e-10
 MMD-DK: 8 MMD:  3.279210068285465e-05mmd2:  5.979236448183656e-06 	 var:  2.4005238303802366e-11
 MMD-DK: 8 MMD:  5.979236448183656e-06mmd2:  8.55035

  0%|          | 0/1000 [00:00<?, ?it/s]

mmd2:  -9.501539170742035e-05 	 var:  5.310666892910376e-06
mmd2:  0.00015288032591342926 	 var:  1.1653180990833789e-05
mmd2:  0.00012441398575901985 	 var:  9.847877663560212e-06
mmd2:  0.00014861347153782845 	 var:  1.1681193427648395e-05
mmd2:  0.0001563667319715023 	 var:  1.4722034393344074e-05
mmd2:  2.7795322239398956e-06 	 var:  7.888949767220765e-06
mmd2:  0.0004232097417116165 	 var:  3.6618301237467676e-05
mmd2:  9.419210255146027e-05 	 var:  6.844988092780113e-06
mmd2:  1.0007526725530624e-05 	 var:  1.0947289410978556e-05
mmd2:  8.51089134812355e-05 	 var:  9.777933883015066e-06
mmd2:  0.00014300737529993057 	 var:  9.925730410031974e-06
mmd2:  0.0002872804179787636 	 var:  2.389666042290628e-05
mmd2:  0.00018343515694141388 	 var:  1.2997639714740217e-05
mmd2:  0.00017143599689006805 	 var:  1.5308891306631267e-05
mmd2:  0.0002113720402121544 	 var:  1.3485303497873247e-05
mmd2:  0.00030031008645892143 	 var:  2.3562002752441913e-05
mmd2:  0.0001445012167096138 	 var:  1

  0%|          | 0/100 [00:00<?, ?it/s]

mmd2:  6.787420716136694e-06 	 var:  4.450280039255586e-11
 MMD-DK: 0 MMD:  6.787420716136694e-06mmd2:  1.1402240488678217e-05 	 var:  4.531981535242569e-11
 MMD-DK: 1 MMD:  1.1402240488678217e-05mmd2:  1.1419644579291344e-05 	 var:  4.1678836813546574e-11
 MMD-DK: 2 MMD:  1.1419644579291344e-05mmd2:  7.813883712515235e-06 	 var:  3.0670154627387945e-11
 MMD-DK: 3 MMD:  7.813883712515235e-06mmd2:  7.639988325536251e-06 	 var:  4.2489349164781124e-11
 MMD-DK: 4 MMD:  7.639988325536251e-06mmd2:  1.5033525414764881e-05 	 var:  3.6064539519262984e-11
 MMD-DK: 5 MMD:  1.5033525414764881e-05mmd2:  1.1706550139933825e-05 	 var:  5.3747515530962235e-11
 MMD-DK: 6 MMD:  1.1706550139933825e-05mmd2:  1.1056603398174047e-05 	 var:  4.985149785605185e-11
 MMD-DK: 7 MMD:  1.1056603398174047e-05mmd2:  6.156507879495621e-06 	 var:  1.7781118054010672e-11
 MMD-DK: 7 MMD:  6.156507879495621e-06mmd2:  1.426198286935687e-05 	 var:  5.2735043030937626e-11
 MMD-DK: 8 MMD:  1.426198286935687e-05mmd2:  1.8999

#### Epoch 20 / lr: 0.0005
[[0.23 0.94 0.25 0.37 0.6  0.1  0.36 0.31 0.23 0.4 ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]]
Average Test Power of Baselines (10 times): 
MMD-D:  0.379

In [11]:
np.mean([0.25, 0.93, 0.94, 0.68, 0.8, 0.6, 0.93, 0.77, 0.78, 0.83])

0.7510000000000001